In [51]:
import pandas as pd
import numpy as np

In [52]:
# Loading data
churn = pd.read_csv('daily-happiness-employee-turnover/churn.csv')
interactions = pd.read_csv('daily-happiness-employee-turnover/commentInteractions.csv')
comments = pd.read_csv('daily-happiness-employee-turnover/comments_clean_anonimized.csv')
votes = pd.read_csv('daily-happiness-employee-turnover/votes.csv')

In [53]:
# Collecting the unique list of companies
companies = pd.Series(churn['companyAlias'].unique())

In [54]:
# Since the companyAlias is too long, lets just replace it with an index for easy identification of company 
churn['companyAlias'] = churn['companyAlias'].apply(lambda x: companies.values.tolist().index(x) if x in companies.values else -1)
interactions['companyAlias'] = interactions['companyAlias'].apply(lambda x: companies.values.tolist().index(x) if x in companies.values else -1)
comments['companyAlias'] = comments['companyAlias'].apply(lambda x: companies.values.tolist().index(x) if x in companies.values else -1)
votes['companyAlias'] = votes['companyAlias'].apply(lambda x: companies.values.tolist().index(x) if x in companies.values else -1)

In [55]:
# Employees are unique only with the company, so lets create a unique employee id
votes['employee'] = votes['companyAlias'].astype(str) + "_" + votes['employee'].astype(str)
churn['employee'] = churn['companyAlias'].astype(str) + "_" + churn['employee'].astype(str)
interactions['employee'] = interactions['companyAlias'].astype(str) + "_" + interactions['employee'].astype(str)
comments['employee'] = comments['companyAlias'].astype(str) + "_" + comments['employee'].astype(str)

In [56]:
# We need to clean the dates - contains CET and CEST text
churn['lastParticipationDate'] = churn['lastParticipationDate'].str.replace('CET','').str.replace('CEST','')
churn['lastParticipationDate'] = pd.to_datetime(churn['lastParticipationDate'],format="%a %b %d %H:%M:%S %Y")

comments['commentDate'] = comments['commentDate'].str.replace('CET','').str.replace('CEST','')
comments['commentDate'] = pd.to_datetime(comments['commentDate'],format="%a %b %d %H:%M:%S %Y")

votes['voteDate'] = votes['voteDate'].str.replace('CET','').str.replace('CEST','')
votes['voteDate'] = pd.to_datetime(votes['voteDate'],format="%a %b %d %H:%M:%S %Y")

In [58]:
# Convert stillExists True/False to 1/0
churn.loc[:,['stillExists']] = churn['stillExists'].astype(int)
# Store only date. Ignoring time for now
churn['lastParticipationDate'] = churn['lastParticipationDate'].dt.date
# Drop numVotes as we can re calculate after merging with daily votes
churn.drop('numVotes', axis=1, inplace=True)
churn.head()

,employee,companyAlias,lastParticipationDate,stillExists
0,0_512,0,2017-02-23,1
1,0_-2,0,2017-01-18,0
2,0_2,0,2017-03-17,1
3,0_487,0,2016-11-19,0
4,0_3,0,2017-02-16,1


In [59]:
# Store only date. Ignoring time for now
votes['voteDate'] = votes['voteDate'].dt.date
votes.head()

,employee,companyAlias,voteDate,vote
0,0_31,0,2016-02-01,4
1,0_33,0,2016-02-01,4
2,0_79,0,2016-02-01,4
3,0_94,0,2016-02-01,4
4,0_16,0,2016-02-01,2


In [87]:
# Merging votes and churn data
votes_churn = pd.merge(votes, churn, left_on=  ['employee','companyAlias','voteDate'], right_on= ['employee','companyAlias','lastParticipationDate'], how = 'left')
votes_churn.drop('lastParticipationDate', axis=1, inplace=True)
votes_churn = votes_churn.fillna(1)
votes_churn = votes_churn.rename(columns={'voteDate':'date'})
votes_churn.drop_duplicates(subset=['employee', 'companyAlias', 'date'], keep='first', inplace=True)
votes_churn.head()

,employee,companyAlias,date,vote,stillExists
0,0_31,0,2016-02-01,4,1.0
1,0_33,0,2016-02-01,4,1.0
2,0_79,0,2016-02-01,4,1.0
3,0_94,0,2016-02-01,4,1.0
4,0_16,0,2016-02-01,2,1.0


In [61]:
# Store only date. Ignoring time for now
comments['commentDate'] = comments['commentDate'].dt.date
# txt column is hardly useful. Maybe length of comment might be a useful feature
# But we have a lot of NANs in txt. 
# So lets assign those txt as blank as comment could have been emoji or gif because it had likes and dislikes
comments = comments.dropna(subset=['commentDate'])
comments['txt'] = comments['txt'].fillna('')
comments['txt'] = comments['txt'].apply(lambda x: len(x))
comments.head()

,employee,companyAlias,commentId,txt,likes,dislikes,commentDate
0,0_307,0,58d018d7e010990004e38070,62,4.0,0.0,2017-03-20
1,0_382,0,58d0179ae010990004e3806d,29,1.0,2.0,2017-03-20
2,0_172,0,58cff8cde010990004e37f6a,27,3.0,0.0,2017-03-20
3,0_135,0,58cfefeee010990004e37f60,27,1.0,1.0,2017-03-20
4,0_225,0,58cfd9b4e010990004e37f52,33,3.0,2.0,2017-03-20


In [95]:
# Merging with comments data
votes_churn_comments = pd.merge(votes_churn, comments, left_on=  ['employee','companyAlias','date'], right_on= ['employee','companyAlias','commentDate'], how = 'left')
votes_churn_comments.drop('commentDate', axis=1, inplace=True)
votes_churn_comments.drop('commentId', axis=1, inplace=True)
votes_churn_comments.drop_duplicates(subset=['employee', 'companyAlias', 'date'], keep='first', inplace=True)
votes_churn_comments = votes_churn_comments.fillna(0)
votes_churn_comments.head()

,employee,companyAlias,date,vote,stillExists,txt,likes,dislikes
0,0_31,0,2016-02-01,4,1.0,48.0,2.0,0.0
2,0_33,0,2016-02-01,4,1.0,89.0,11.0,0.0
4,0_79,0,2016-02-01,4,1.0,34.0,4.0,0.0
6,0_94,0,2016-02-01,4,1.0,11.0,2.0,0.0
8,0_16,0,2016-02-01,2,1.0,0.0,0.0,0.0


In [96]:
# Drop where companyAlias is -1
interactions = interactions[interactions.companyAlias!=-1]
# Convert True/False to 1/0
interactions.loc[:,['liked']] = interactions['liked'].astype(int)
interactions.loc[:,['disliked']] = interactions['disliked'].astype(int)
interactions.head()

,employee,companyAlias,liked,disliked,commentId
0,0_307,0,1,0,58d018d7e010990004e38070
1,0_36,0,1,0,58d018d7e010990004e38070
2,0_276,0,1,0,58d018d7e010990004e38070
3,0_24,0,1,0,58d018d7e010990004e38070
4,0_382,0,1,0,58d0179ae010990004e3806d


In [97]:
# How social are employees? How many times they have liked, disliked
emp_interactions_1 = interactions.groupby('employee')\
    .aggregate({'liked': 'sum','disliked': 'sum'})\
    .reset_index()
emp_interactions_1 = emp_interactions_1.rename(columns={'liked':'total_liked', 'disliked':'total_disliked'})
emp_interactions_1.head()

,employee,total_liked,total_disliked
0,0_-2,3,0
1,0_-4,4,2
2,0_-6,2,0
3,0_10,29,6
4,0_100,288,2


In [98]:
# How social are employees? How many times they have commented, received likes, dislikes
emp_interactions_2 = comments.groupby('employee')\
    .aggregate({'commentId': 'count','txt': 'mean','likes':'sum','dislikes':'sum'})\
    .reset_index()
emp_interactions_2 = emp_interactions_2.rename(columns={'commentId':'total_comments', 'txt':'avg_comment_len', 'likes':'likes_received', 'dislikes':'dislikes_received'})
emp_interactions_2.head()

,employee,total_comments,avg_comment_len,likes_received,dislikes_received
0,0_-3,1,92.000000,6.0,0.0
1,0_-4,22,54.181818,116.0,46.0
2,0_-5,13,202.692308,66.0,72.0
3,0_-7,3,117.333333,10.0,1.0
4,0_10,6,86.333333,16.0,20.0


In [99]:
# Joining both the data sets to get all social activities of employee
emp_social_data = pd.merge(emp_interactions_1, emp_interactions_2, on=['employee'], how = 'left')
# Missing ones those who just never wrote a comment. Hence imputing zero
emp_social_data = emp_social_data.fillna(0)

In [59]:
emp_social_data.head()

,employee,total_liked,total_disliked,total_comments,avg_comment_len,likes_received,dislikes_received
0,0_-2,3,0,0.0,0.000000,0.0,0.0
1,0_-4,4,2,22.0,54.181818,116.0,46.0
2,0_-6,2,0,0.0,0.000000,0.0,0.0
3,0_10,29,6,6.0,86.333333,16.0,20.0
4,0_100,288,2,60.0,64.433333,326.0,50.0


In [101]:
# How happy are employees?
emp_votes = votes.groupby('employee')\
    .aggregate({'vote': 'mean'})\
    .rename(columns={'vote':'avg_vote'})\
    .reset_index()
emp_votes['avg_vote'] = emp_votes['avg_vote'].apply(lambda x: int(round(x)))

In [102]:
emp_votes.head()

,employee,avg_vote
0,0_10,3
1,0_100,4
2,0_101,3
3,0_102,4
4,0_103,4


In [105]:
votes_churn_comments.to_csv("Master_Data_new.csv", index=False)